In [1]:
import ee
ee.Initialize()
import folium
import geehydro


In [2]:
def gapFillAndExtendBounds(multiBandImage,bandList,distanceToFillInMeters,categorical):
    
    # N.B., This script is based on code from Matt Hancher and includes annotations
    # from his original example code.
    # Use a helper function to fill holes and boundaries with the nearest value
    def fillHolesWithNearestValue(imageToFill):
        source = imageToFill.mask();
        # Measure 1000000x the distance to the nearest valid pixel. We pick
        # a large cost (here 1000000) so that the least-cost path will not be
        # influenced later by the border pixels.
        cost0 = ee.Image(1000000).where(source, 0).cumulativeCost(source, distanceToFillInMeters);
        
        # Measure the distance to the nearest pixel plus the half-pixel
        # traversal to the center of a valid border pixel, which may be
        # 1/2 or 1/sqrt(2).
        cost1 = ee.Image(1000000).where(source, 1).cumulativeCost(source, distanceToFillInMeters);
        
        # Measure the distance to the nearest pixel plus the half-pixel
        # traversal to center of a valid pixel, where the valid pixel
        # has a cost equal to its original value.
        cost2 = imageToFill.unmask(1000000).cumulativeCost(source, distanceToFillInMeters);
        
        # Finally we can compute the original value of the neares
        # unmasked pixel.
        fill = cost2.subtract(cost0).divide(cost1.subtract(cost0));
        
        # Fill in the masked pixels.
        filled = imageToFill.unmask(0).add(fill);
        
        return filled.copyProperties(imageToFill);
        
    # Use a helper function to convert an image collection to a multiband image
    def icToImage(imageCollection):
        
        # Create an empty image to fill
        emptyImage = ee.Image(0).rename('BAND_TO_REMOVE');
        
        # Iterate through the collection to make the new multiband image
        def functionToIterate(image, result):
            return ee.Image(result).addBands(image)
        multibandImageToSub = ee.Image(imageCollection.iterate(functionToIterate, emptyImage));
        multibandImageSelected = multibandImageToSub.select(multibandImageToSub.bandNames().remove('BAND_TO_REMOVE'))
        
        return multibandImageSelected;
        
    # Turn the multiband image of interest into an image collection
    imageCollection = ee.ImageCollection(multiBandImage.bandNames().map(lambda bandName: multiBandImage.select([bandName]).set('imageName',bandName)));
    # print('Image Collection from Composite',imageCollection);
    
    # Separate out the images that shouldn't be filled
    imagesNotToFill = imageCollection.filter(ee.Filter.inList('imageName',bandList).Not());
    # print('Images not to Fill',imagesNotToFill);
    
    imagesNotFilled = icToImage(imagesNotToFill);
    # print('Images not Filled',imagesNotFilled);
    
    imagesToFill = imageCollection.filter(ee.Filter.inList('imageName',bandList));
    # print('Images to Fill',imagesToFill);
    
    imagesFilled = imagesToFill.map(fillHolesWithNearestValue);
    # print('Gap Filled Images',imagesFilled);
    
    multiBandImageFilled = icToImage(imagesFilled);
    # print('Multiband image filled',multiBandImageFilled);
    
    stackedCompositeFilled = ee.Image.cat(multiBandImageFilled,imagesNotFilled);
    # print('Filled Composite',stackedCompositeFilled);

    # Return a rounded version of the image if it is categorical
    if categorical == True:
        # If it's a categorical data type, round the image to the nearest integer
        imageToReturn = stackedCompositeFilled.round()
    else:
        # If it's continuous, don't round the value
        imageToReturn = stackedCompositeFilled
        
    return imageToReturn;

In [3]:
# Load the example WWF Biome Image
exampleImage = ee.Image("users/devinrouth/DELETEME_GapFilling_Sampling_Practice_Image");


In [4]:
# Define the list of bands to fill / extend
listOfBandsToFill = ['WWF_Biome'];
filledImage = gapFillAndExtendBounds(exampleImage,listOfBandsToFill,10000,True);


In [5]:
# See how the object prints
print(filledImage.getInfo())


{'type': 'Image', 'bands': [{'id': 'WWF_Biome', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [317, 166], 'crs': 'EPSG:4326', 'crs_transform': [0.008333333333333333, 0, 4.383333333333326, 0, -0.008333333333333333, 59.91666666666667]}]}


In [6]:
# View it on a map
Map = folium.Map(location=[59.2194809093723,5.8477799483598725], zoom_start=7)
Map.setOptions('HYBRID')
# Map.addLayer(exampleImage.randomVisualizer())
Map.addLayer(filledImage.randomVisualizer())
Map


In [7]:
# Input the exporting parameters
geoForExport = ee.Geometry.Polygon(
        [[[4.38351392495405, 59.91633906274373],
          [4.38351392495405, 58.53944442726217],
          [7.02023267495405, 58.53944442726217],
          [7.02023267495405, 59.91633906274373]]], None, False);
pyramidingPolicy = "mode"
description = 'DELETEME_GapFilling_Python_Test'


In [8]:
# Define the export
bootstrapImageExport = ee.batch.Export.image.toAsset(
    image=filledImage,
    description=description,
    assetId='users/devinrouth/'+description,
    crs='EPSG:4326',
    crsTransform='[0.008333333333333333,0,-180,0,-0.008333333333333333,90]',
    region=geoForExport.getInfo()['coordinates'],
    maxPixels=int(1e13),
    pyramidingPolicy={".default": pyramidingPolicy}
);


In [9]:
# Start the export
bootstrapImageExport.start()
